In [23]:
import numpy as np
import os
import trimesh
from scipy.spatial import KDTree

import numpy as np
import glob

import matplotlib
matplotlib.use('agg')
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools
from collections import defaultdict
import pandas as pd
import networkx as nx
from pathlib import Path
import sys
sys.setrecursionlimit(100000)


sys.path.append('..')

from notebooks.io import write_ply

In [2]:
from io import BytesIO
import yaml
from yaml import Loader
import k3d
import randomcolor

In [3]:
import sys
sys.path.append('/home/artonson/repos/sharp_features/')

from sharpf.data.abc_data import ABCChunk

In [4]:
def trimesh_load(io: BytesIO):
    """Read the mesh: since trimesh messes the indices, this has to be done manually."""

    vertices, faces = [], []

    for line in io.read().decode('utf-8').splitlines():
        values = line.strip().split()
        if not values: continue
        if values[0] == 'v':
            vertices.append(np.array(values[1:4], dtype='float'))
        elif values[0] == 'f':
            faces.append(np.array([values[1].split('//')[0], values[2].split('//')[0], values[3].split('//')[0]],
                                  dtype='int'))

    vertices = np.array(vertices)
    faces = np.array(faces) - 1

    mesh = trimesh.base.Trimesh(vertices=vertices, faces=faces, process=False)  # create a mesh from the vertices
    return mesh

In [5]:
chunk = ABCChunk(['/home/artonson/tmp/abc/abc_0000_obj_v00.7z', '/home/artonson/tmp/abc/abc_0000_feat_v00.7z'])

In [6]:
item = chunk.get('00002992_79022290f16b4fd9a0a85b2d_000')

In [7]:
mesh = trimesh_load(item.obj)
labels = yaml.load(item.feat, Loader=Loader)

In [ ]:
# import yaml
# import pywavefront
# from yaml import CLoader as Loader, CDumper as Dumper

# def get_config(path):
#     with open(path, 'r') as stream:
#         config = yaml.load(stream, Loader=Loader)
#     return config

In [27]:
shapes = {}

shapes['00002992_79022290f16b4fd9a0a85b2d_000'] = {
    'vertices': process_vertices(np.array(mesh.vertices)),
    'faces': mesh.faces,
    'labels': labels,
    'edges': mesh.edges
}
shapes_ids = ['00002992_79022290f16b4fd9a0a85b2d_000']

In [29]:
plot = k3d.plot(height=768)

k3d_points = k3d.points(mesh.vertices, point_size=0.5, color=0x0000ff)
plot += k3d_points
k3d_points.shader='3d'

# sharp_points = mesh.vertices[np.array(list(shapes[shape_id]['sharp_vertices']))]
# k3d_points = k3d.points(sharp_points, point_size=0.5, color=0xff0000)
# plot += k3d_points
# k3d_points.shader='3d'


plot.grid_visible = False
# plot.display()

/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


## Preprocessing

In [30]:

def process_vertices(vertices):
    '''Centering and scaling'''
    centroid = vertices.mean(axis=0)
#     print(centroid)
    centered = vertices-centroid
    max_dim = abs(centered.max(axis=0) - centered.min(axis=0))
#     print(max_dim)
    max_dim = max(max_dim)
    centered/=max_dim
    return centered
    
    

In [31]:
# shapes_ids = [ '00000005', '00000212']#,'00000416']#'00000005']# ]#'00000005', '00000212','00000416',

# data_path = Path('/home/user/data/abc_new')
# shapes = {}
# for shape_id in tqdm(shapes_ids):

#     mesh_file = glob.glob((data_path/shape_id/'*.obj').as_posix())[0]

#     labels_file = glob.glob((data_path/shape_id/'*features*.yml').as_posix())[0]
# #     print('labels_file', labels_file)
#     labels = get_config(labels_file)

#     scene = pywavefront.Wavefront(mesh_file, collect_faces=True)
#     vertices = process_vertices(np.array(scene.vertices))
#     faces = scene.mesh_list[0].faces
#     edges = set()
#     for face in faces:
#         for edge in itertools.combinations(face, 2):
#             edges.add(edge)

# #     print('edges')
# #     mesh = trimesh.base.Trimesh(vertices=vertices,
# #                                 faces=faces,
# #                                 process=False,
# #                                 validate=False,)

#     shapes[shape_id] = {'vertices': vertices, 'faces': faces, 'labels': labels, 'edges': edges }# , 'mesh': mesh}

In [32]:
def get_vertices_from_face_indices(face_indices, faces):
    face_vertices = set()
    face_dict = defaultdict(list)
    
    for face_index in face_indices:
        face_vertices.update(faces[face_index])
        
        for vert in faces[face_index]:
            face_dict[vert].append(face_index)
            
    return list(face_vertices), face_dict

In [33]:
for shape_id in tqdm(shapes_ids):    

    sharp_vertices = set()
    sharp_edges = defaultdict(set)
    all_sharp_edges = set()
    shape_curves = defaultdict(set)
    for i, curv in enumerate(shapes[shape_id]['labels']["curves"]):
        if curv["sharp"]:
            vert_ind = np.array(curv["vert_indices"]) #+ 1

            sharp_vertices.update(vert_ind)
            shape_curves[i] = vert_ind
            for ind1, ind2 in itertools.combinations(vert_ind, 2):
                if (ind1, ind2) in shapes[shape_id]['edges'] or (ind2, ind1) in shapes[shape_id]['edges']:
                    sharp_edges[i].add((ind1, ind2))
                    all_sharp_edges.add((ind1, ind2))
    
   
    shapes[shape_id]['sharp_vertices'] = sharp_vertices
    shapes[shape_id]['sharp_edges'] = sharp_edges  # dict by curve
    shapes[shape_id]['all_sharp_edges'] = all_sharp_edges # all
    shapes[shape_id]['shape_curves'] = shape_curves
            

100%|██████████| 1/1 [00:23<00:00, 23.91s/it]


In [35]:
plot = k3d.plot(height=768)
rand_color = randomcolor.RandomColor()

k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb)
plot += k3d_mesh

# k3d_points = k3d.points(mesh.vertices, point_size=0.5, color=0x0000ff)
# plot += k3d_points
# k3d_points.shader='3d'

# sharp_points = mesh.vertices[np.array(list(shapes[shape_id]['sharp_vertices']))]
# k3d_points = k3d.points(sharp_points, point_size=0.5, color=0xff0000)
# plot += k3d_points
# k3d_points.shader='3d'

for i, vert_ind in shape_curves.items():
    sharp_points_curve = mesh.vertices[vert_ind]
    color = rand_color.generate(hue='red')[0]
    plt_line = k3d.line(sharp_points_curve, shader='mesh', width=0.5, color=int('0x' + color[1:], 16))
    plot += plt_line

plot.grid_visible = False
plot.display()

/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/artonson/.pyenv/versions/3.6.6/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## Mapping vertices to all faces it belongs to

In [ ]:
for shape_id in shapes_ids: 
    face_dict = defaultdict(list)
    for i, surface in enumerate(tqdm(shapes[shape_id]['labels']["surfaces"])):
        vertices, surface_face_dict = get_vertices_from_face_indices(surface['face_indices'], shapes[shape_id]['faces'])
        for vertex in surface_face_dict:        
            face_dict[vertex]+=surface_face_dict[vertex]
    shapes[shape_id]['face_dict'] = face_dict

## (Almost) spherical patch sampling

In [21]:
def random_spherical_patch(shape, k, radius, root=None):
    '''
    samples mesh patch
    k: pick k vertices within radius
    radius: sphere radius
    '''
    # root vertex to sample around
    if root is None:
        root = np.random.choice(len(shape['vertices']))
        
    start = time.time()
    # tree to sample from
    tree = KDTree(shape['vertices'])
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  building KD tree", timing=end - start))
    
    start = time.time()
    distances, vert_indices = tree.query(shape['vertices'][root], k=k, distance_upper_bound=radius)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  query in KD tree", timing=end - start))

    start = time.time()
    vertices = set()
    faces_indices = []
    for vert_index in vert_indices:
        vert_faces = shape['face_dict'][vert_index]
        for face in vert_faces:
            # if any vertex from a face is inside the sphere pick this face
            if len(np.intersect1d(shape['faces'][face], list(vert_indices)))>0:
                faces_indices.append(face)
                vertices.update(shape['faces'][face])
    vertices = list(vertices)   
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  build vertices", timing=end - start))

#     print(vertices)
    start = time.time()
    reindex = dict(zip(vertices, range(len(vertices))))
    faces = []
    
#     print(faces_indices)
    for face in np.array(shape['faces'])[faces_indices]:
        faces.append([reindex[vert] for vert in face])
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  reindex", timing=end - start))
    
    
    start = time.time()
    mesh = trimesh.base.Trimesh(vertices=shape['vertices'][vertices],
                                faces=faces,
                                process=False,
                                validate=False,)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  create Trimesh", timing=end - start))
    
    return mesh, vertices, faces_indices


def resample(shape, vert_indices, face_indices, patch, target_dist=0.6, n_points=1000, distance_upper_bound=1.0, \
             noise_ampl=0.05):
    '''
    sample annotated point cloud from a mesh patch
    target_dist: desired distance between points on the borders (should match point cloud density)
    distance_upper_bound: maximum distance within which annotation is comuted
    noise_ampl: noise amplitude to shutter the regular samples on the border (depends from target_dist)
    
    '''
    sharp_samples = []
    
    # surface point samples
    start = time.time()
    point_samples, index = trimesh.sample.sample_surface(patch, n_points)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  sample_surface", timing=end - start))

    start = time.time()
    pc_tree = KDTree(point_samples)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  build KDTree on sampled points", timing=end - start))

    start = time.time()
    for vert1, vert2 in itertools.combinations(vert_indices, 2):
#         vert1 in shape['sharp_vertices'] and vert2 in shape['sharp_vertices'] and 
        if ((vert1, vert2) in shape['all_sharp_edges'] or (vert2, vert1) in shape['all_sharp_edges']):
            v1 = shape['vertices'][vert1]
            v2 = shape['vertices'][vert2]
            
            # quite an ugly way to estimate the number of points to sample on edge
            n_samples = 2+int(np.linalg.norm(v1-v2)/target_dist+0.5)

            # border samples from sharp edges
            sharp_samples.append(sample_even(v1, v2, n_samples))
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  sample sharp edges", timing=end - start))
    
    # patch without sharp features
    if len(sharp_samples)==0:
        return point_samples, np.ones_like(point_samples[:,0])*distance_upper_bound, np.zeros_like(point_samples)
    sharp_samples = np.concatenate(sharp_samples)  
    
    ### !!! kostile
    n_sharp = len(sharp_samples)
    idx_to_keep = np.random.choice(np.arange(n_points), 
                                   size=n_points - n_sharp, replace=False)
    point_samples = point_samples[idx_to_keep]
    ### !!! end kostile
    
    # computing distances to the border samples
    start = time.time()
    tree = KDTree(sharp_samples)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  build KD tree on sharp samples", timing=end - start))

    start = time.time()
    distances, vert_indices = tree.query(point_samples, distance_upper_bound=distance_upper_bound)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  query KD tree on sharp samples", timing=end - start))

#     print(distances[:40], vert_indices[:40])
#     print(len(distances), len(vert_indices), vert_indices.max(), len(sharp_samples))
    start = time.time()
    mask_point = distances==np.inf
    mask_sharp = vert_indices<len(sharp_samples)
    vert_indices = vert_indices[mask_sharp]
    
    directions = np.zeros_like(point_samples)
    directions[~mask_point] = sharp_samples[vert_indices] - point_samples[~mask_point]

    distances[mask_point] = distance_upper_bound
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  form distances", timing=end - start))

    
    # adding noise to regular boundary samples
    start = time.time()

    noise = (np.random.rand(*sharp_samples.shape)-0.5)*2*noise_ampl
    edge_samples = sharp_samples + noise
    vert = np.concatenate([point_samples, sharp_samples])
    distances = np.concatenate([distances, np.linalg.norm(noise, axis=1)])
    directions = np.concatenate([directions, noise])
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="  add noise", timing=end - start))

    return vert, distances, directions

def sample_even(v1, v2, count, noise_ampl=0.005):
    unit = np.linspace(0, 1, count)[...,None] + noise_ampl*np.random.rand(count, 1)
    unit = np.clip(unit, 0, 1)

    vec = (v2-v1)[None, ...]
    
    return v1 + vec*unit

In [ ]:
save_dir = 'curves_test'
os.makedirs(save_dir, exist_ok=True)

## Testing

In [ ]:
num_samples = 1


# patch mesh sampling
n_vert = 400 # number of mesh vertices to pick
max_dist = 1 # sphere radius

# pc sampling
distance_upper_bound = 0.01 # distance around borders 
target_dist=0.00001 # distance between regular edge samples
n_points=4096 
noise_ampl = 5e-4 # noise amplitude for regular edge samples


patches = []
for shape_id in shapes_ids:

    for i in tqdm(range(num_samples)):
        mesh, vert_indices, faces_indices = random_spherical_patch(shapes[shape_id], n_vert, max_dist)
        
        patches.append((mesh, vert_indices, faces_indices))
#         target_dist = np.sqrt(mesh.area_faces.sum()/n_points)/10
        noise_ampl = 0 #np.sqrt(target_dist)*1e-3
#         print(target_dist)

        point_samples, distances, directions = resample(
            shapes[shape_id], vert_indices, faces_indices, mesh, n_points=n_points,
            distance_upper_bound=distance_upper_bound, noise_ampl=noise_ampl)
        write_ply(f'{save_dir}/{shape_id}_{i}.ply', point_samples, distances=distances/distance_upper_bound*255, directions=directions)

In [ ]:
distances.shape

In [25]:
from IPython.display import Image
from base64 import b64decode

def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='3d'

    if None is not samples:
        if None is not samples_distances:
            max_dist = np.max(distances)
            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.basic_color_maps.WarmCool, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
            
        plot += k3d_points
        k3d_points.shader='3d'

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='3d'

        rand_color = randomcolor.RandomColor()
        for i, vert_ind in shape_curves.items():
            sharp_points_curve = mesh.vertices[vert_ind]
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                color = rand_color.generate(hue='red')[0]
                color = int('0x' + color[1:], 16)
            plt_line = k3d.line(sharp_points_curve, 
                                shader='mesh', width=sharpcurve_width, color=color)
            plot += plt_line

    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [26]:
im = display_sharpness(mesh=mesh, samples=point_samples, samples_distances=distances, as_image=True)

NameError: name 'point_samples' is not defined

In [ ]:
plot = k3d.plot(height=768)
rand_color = randomcolor.RandomColor()

k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb)
plot += k3d_mesh

k3d_points = k3d.points(mesh.vertices, point_size=0.0025, color=0x666666)
plot += k3d_points
k3d_points.shader='3d'


colors = k3d.helpers.map_colors(
    distances, k3d.colormaps.basic_color_maps.WarmCool,[0,0.01]).astype(np.uint32)

k3d_points = k3d.points(point_samples, point_size=0.002, colors=colors)
plot += k3d_points
k3d_points.shader='3d'

# sharp_points = mesh.vertices[np.array(list(shapes[shape_id]['sharp_vertices']))]
# k3d_points = k3d.points(sharp_points, point_size=0.5, color=0xff0000)
# plot += k3d_points
# k3d_points.shader='3d'

# for i, vert_ind in shape_curves.items():
#     sharp_points_curve = mesh.vertices[vert_ind]
#     color = rand_color.generate(hue='red')[0]
#     plt_line = k3d.line(sharp_points_curve, shader='mesh', width=0.5, color=int('0x' + color[1:], 16))
#     plot += plt_line

plot.grid_visible = False
# plot.camera_auto_fit = True
plot.camera = [1.5, 1.5, 1.5, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
plot.display()

In [ ]:
from IPython.display import Image
from base64 import b64decode
i = Image(data=b64decode(plot.screenshot))

In [ ]:
plot.fetch_screenshot()

In [ ]:
Image(data=b64decode(plot.screenshot))

In [ ]:
plot = k3d.plot(height=768)
rand_color = randomcolor.RandomColor()

k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=0xbbbbbb)
plot += k3d_mesh

k3d_points = k3d.points(mesh.vertices, point_size=0.0025, color=0x0000ff)
plot += k3d_points
k3d_points.shader='3d'

# sharp_points = mesh.vertices[np.array(list(shapes[shape_id]['sharp_vertices']))]
# k3d_points = k3d.points(sharp_points, point_size=0.5, color=0xff0000)
# plot += k3d_points
# k3d_points.shader='3d'

# for i, vert_ind in shape_curves.items():
#     sharp_points_curve = mesh.vertices[vert_ind]
#     color = rand_color.generate(hue='red')[0]
#     plt_line = k3d.line(sharp_points_curve, shader='mesh', width=0.5, color=int('0x' + color[1:], 16))
#     plot += plt_line

plot.grid_visible = False
plot.display()

In [32]:
def curves(shape):
    sharp_samples = []
    
    for curve in shape['shape_curves']:
#         print(shape['shape_curves'][curve])
        sharp_samples+=list(shape['shape_curves'][curve])
    
    vert = shape['vertices'][sharp_samples]#np.concatenate(sharp_samples)  
    distances = np.zeros_like(vert[:,0])# np.concatenate([distances, np.linalg.norm(noise, axis=1)])
    
    return vert, distances

In [ ]:
item.item_id

### Load the whole chunk and process it into patches

In [13]:
import time

print("hello")


In [11]:
import h5py

In [19]:
def process_parallel(meshes_filename, feats_filename, data_slice):
    
    num_samples_per_shape = 10

    # patch mesh sampling
    n_vert = 400 # number of mesh vertices to pick
    max_dist = 1 # sphere radius

    # pc sampling
    distance_upper_bound = 0.01 # distance around borders 
    target_dist=0.00001 # distance between regular edge samples
    n_points=4096 
    noise_ampl = 5e-4 # noise amplitude for regular edge samples


    slice_start, slice_end = data_slice
    
    start = time.time()
    chunk = ABCChunk([meshes_filename, feats_filename])
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="Load chunk", timing=end - start))


    point_patches, point_distances, point_directions = [], [], []
    
    for item in chunk[slice_start:slice_end]:
        start = time.time()
        mesh = trimesh_load(item.obj)
        labels = yaml.load(item.feat, Loader=Loader)
        end = time.time()
        print("{action}: {timing:3.3f}".format(action="Load mesh from chunk", timing=end - start))


        
        # form what's needed to sample
        start = time.time()

        shapes = {}
        shapes_ids = [item.item_id]
        shapes[item.item_id] = {
            'vertices': process_vertices(np.array(mesh.vertices)),
            'faces': mesh.faces,
            'labels': labels,
            'edges': mesh.edges
        }
        end = time.time()
        print("{action}: {timing:3.3f}".format(action="Form shape (process_vertices)", timing=end - start))

        
        
        start = time.time()
        face_dict = defaultdict(list)
        for i, surface in enumerate(shapes[item.item_id]['labels']["surfaces"]):
            vertices, surface_face_dict = get_vertices_from_face_indices(
                surface['face_indices'], shapes[item.item_id]['faces'])
            for vertex in surface_face_dict:        
                face_dict[vertex] += surface_face_dict[vertex]
        shapes[item.item_id]['face_dict'] = face_dict
        end = time.time()
        print("{action}: {timing:3.3f}".format(action="Form shape (get_vertices_from_face_indices)", timing=end - start))


            
        start = time.time()
        sharp_vertices = set()
        sharp_edges = defaultdict(set)
        all_sharp_edges = set()
        shape_curves = defaultdict(set)
        for i, curv in enumerate(shapes[item.item_id]['labels']["curves"]):
            if curv["sharp"]:
                vert_ind = np.array(curv["vert_indices"])

                sharp_vertices.update(vert_ind)
                shape_curves[i] = vert_ind
                for ind1, ind2 in itertools.combinations(vert_ind, 2):
                    if (ind1, ind2) in shapes[item.item_id]['edges'] or (ind2, ind1) in shapes[item.item_id]['edges']:
                        sharp_edges[i].add((ind1, ind2))
                        all_sharp_edges.add((ind1, ind2))

            shapes[item.item_id]['sharp_vertices'] = sharp_vertices
            shapes[item.item_id]['sharp_edges'] = sharp_edges  # dict by curve
            shapes[item.item_id]['all_sharp_edges'] = all_sharp_edges # all
            shapes[item.item_id]['shape_curves'] = shape_curves
        end = time.time()
        print("{action}: {timing:3.3f}".format(action="Form shape (sharp_edges)", timing=end - start))

            
            
        # crop patches
        patches = []


        for i in range(num_samples_per_shape):
            try:

                start = time.time()
                nbhood_mesh, vert_indices, faces_indices = random_spherical_patch(
                    shapes[item.item_id], n_vert, max_dist)
                end = time.time()
                print("{action}: {timing:3.3f}".format(action="random_spherical_patch", timing=end - start))

            except RecursionError:
                print ('Recursion (KD tree) error on shape {}'.format(item.item_id))
            else:

                patches.append((mesh, vert_indices, faces_indices))
        #         target_dist = np.sqrt(mesh.area_faces.sum()/n_points)/10
                noise_ampl = 0 #np.sqrt(target_dist)*1e-3
        #         print(target_dist)

                start = time.time()
                point_samples, distances, directions = resample(
                    shapes[item.item_id], vert_indices, faces_indices, nbhood_mesh, n_points=n_points,
                    distance_upper_bound=distance_upper_bound, noise_ampl=noise_ampl)
                end = time.time()
                print("{action}: {timing:3.3f}".format(action="resample", timing=end - start))


                point_patches.append(point_samples)
                point_distances.append(distances)
                point_directions.append(directions)

        
                
        
#                 write_ply(f'{save_dir}/{shape_id}_{i}.ply', point_samples, distances=distances/distance_upper_bound*255, directions=directions)
                

    start = time.time()
    point_patches = np.array(point_patches)  # num_samples_per_shape * num_shapes, 4096, 3
    point_distances = np.array(point_distances)  # num_samples_per_shape * num_shapes, 4096, 1
    point_directions = np.array(point_directions)  # 1num_samples_per_shape * num_shapes, 4096, 3
    
    sharp_dir_norms = np.linalg.norm(point_directions, axis=-1, keepdims=True)
    nonzero = np.squeeze(sharp_dir_norms > 0, axis=-1)
    point_directions[nonzero] /= sharp_dir_norms[nonzero]
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="Postprocessing", timing=end - start))

    
    start = time.time()
    with h5py.File("/home/artonson/tmp/abc/abc0000_{}_{}.hdf5".format(slice_start, slice_end), "w") as f:
        f.create_dataset("data", data=point_patches, dtype=np.float64)
        f.create_dataset("distances", data=point_distances, dtype=np.float64)
        f.create_dataset("directions", data=point_directions, dtype=np.float64)
    end = time.time()
    print("{action}: {timing:3.3f}".format(action="Saving to disk", timing=end - start))



In [22]:
process_parallel('/home/artonson/tmp/abc/abc_0000_obj_v00.7z',
                 '/home/artonson/tmp/abc/abc_0000_feat_v00.7z',
                 (0, 1))

Load chunk: 7.385
Load mesh from chunk: 30.212
Form shape (process_vertices): 0.005
Form shape (get_vertices_from_face_indices): 0.609
Form shape (sharp_edges): 176.276
  building KD tree: 0.216
  query in KD tree: 0.009
  build vertices: 0.601
  reindex: 0.010
  create Trimesh: 0.003
random_spherical_patch: 0.847
  sample_surface: 0.005
  build KDTree on sampled points: 0.038
  sample sharp edges: 0.152
  build KD tree on sharp samples: 0.030
  query KD tree on sharp samples: 0.673
  form distances: 0.000
  add noise: 0.000
resample: 0.906
  building KD tree: 0.203
  query in KD tree: 0.011
  build vertices: 0.468
  reindex: 0.010
  create Trimesh: 0.003
random_spherical_patch: 0.702
  sample_surface: 0.004
  build KDTree on sampled points: 0.042
  sample sharp edges: 0.122
  build KD tree on sharp samples: 0.002
  query KD tree on sharp samples: 0.425
  form distances: 0.001
  add noise: 0.000
resample: 0.600
  building KD tree: 0.110
  query in KD tree: 0.006
  build vertices: 0.340

In [108]:
chunk_size = 100

obj_filename = '/home/artonson/tmp/abc/abc_0000_obj_v00.7z'
feat_filename = '/home/artonson/tmp/abc/abc_0000_feat_v00.7z'
    
abc_data_slices = [(start, start + chunk_size)
                   for start in range(0, 7200, chunk_size)]


In [109]:
from joblib import Parallel, delayed

In [ ]:

# run the filtering job in parallel
delayed_iterable = (delayed(process_parallel)(obj_filename, feat_filename, data_slice)
                    for data_slice in abc_data_slices)

parallel = Parallel(n_jobs=36, backend="loky")
parallel(delayed_iterable)


In [40]:
!hostname

Prometheus
